In [ ]:
path = "/content/drive/MyDrive/5MLRE/IMDB/data/imdb_top_1000.csv"

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.neighbors import NearestNeighbors

In [ ]:
df_0 = pd.read_csv(path)

In [ ]:
df = df_0.copy()

In [ ]:
df.columns

Index(['Poster_Link', 'Series_Title', 'Released_Year', 'Certificate',
       'Runtime', 'Genre', 'IMDB_Rating', 'Overview', 'Meta_score', 'Director',
       'Star1', 'Star2', 'Star3', 'Star4', 'No_of_Votes', 'Gross'],
      dtype='object')

In [ ]:
df

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://m.media-amazon.com/images/M/MV5BNGEwMT...,Breakfast at Tiffany's,1961,A,115 min,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,76.0,Blake Edwards,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen,166544,NaN
996,https://m.media-amazon.com/images/M/MV5BODk3Yj...,Giant,1956,G,201 min,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,84.0,George Stevens,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker,34075,NaN
997,https://m.media-amazon.com/images/M/MV5BM2U3Yz...,From Here to Eternity,1953,Passed,118 min,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",85.0,Fred Zinnemann,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed,43374,"30,500,000"
998,https://m.media-amazon.com/images/M/MV5BZTBmMj...,Lifeboat,1944,NaN,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.0,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,26471,NaN


Description des caractéristiques : https://www.kaggle.com/datasets/inductiveanks/top-1000-imdb-movies-dataset


In [ ]:
# On ne garde que ces colonnes pour le système de recommandation :
cols = ['Genre', 'Released_Year', 'Runtime', 'IMDB_Rating', 'Meta_score', 'No_of_Votes','Gross', 'Director', 'Star1', 'Star2', 'Star3', 'Star4']

In [ ]:
df = df[cols]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Genre          1000 non-null   object 
 1   Released_Year  1000 non-null   object 
 2   Runtime        1000 non-null   object 
 3   IMDB_Rating    1000 non-null   float64
 4   Meta_score     843 non-null    float64
 5   No_of_Votes    1000 non-null   int64  
 6   Gross          831 non-null    object 
 7   Director       1000 non-null   object 
 8   Star1          1000 non-null   object 
 9   Star2          1000 non-null   object 
 10  Star3          1000 non-null   object 
 11  Star4          1000 non-null   object 
dtypes: float64(2), int64(1), object(9)
memory usage: 93.9+ KB


# Prétraitement

## Runtime
Suppression des " min" puis conversion en int

In [ ]:
for i in list(df["Runtime"]):
  if not isinstance(i, int):
    print(i)

142 min
175 min
152 min
202 min
96 min
201 min
154 min
195 min
148 min
139 min
178 min
142 min
161 min
179 min
136 min
146 min
124 min
133 min
160 min
132 min
153 min
169 min
130 min
125 min
169 min
189 min
116 min
127 min
118 min
121 min
133 min
207 min
130 min
122 min
106 min
112 min
130 min
151 min
150 min
155 min
119 min
106 min
110 min
88 min
137 min
155 min
89 min
116 min
165 min
109 min
102 min
87 min
87 min
126 min
125 min
147 min
106 min
161 min
117 min
181 min
149 min
105 min
165 min
164 min
170 min
165 min
98 min
137 min
101 min
113 min
134 min
229 min
115 min
146 min
147 min
117 min
122 min
143 min
95 min
116 min
88 min
112 min
110 min
125 min
119 min
104 min
139 min
160 min
115 min
123 min
131 min
132 min
112 min
153 min
108 min
122 min
104 min
102 min
122 min
126 min
89 min
81 min
178 min
99 min
116 min
142 min
137 min
160 min
170 min
131 min
149 min
114 min
129 min
136 min
149 min
132 min
228 min
125 min
136 min
128 min
103 min
143 min
89 min
107 min
119 min
117 min
153 

In [ ]:
# df.loc[:,"Runtime"] = df["Runtime"].str.replace("min", "").astype("int")
df["Runtime"] = df["Runtime"].str.replace("min", "").astype("int")

<ipython-input-166-2272b3953b48>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Runtime"] = df["Runtime"].str.replace("min", "").astype("int")


## Gross
Suppression des "," puis conversion en float

In [ ]:
df["Gross"] = df["Gross"].str.replace(",", "").astype("float")

<ipython-input-167-132d0aa1d067>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Gross"] = df["Gross"].str.replace(",", "").astype("float")


# Released Year
Une anomalie est présente dans le dataset : "PG" au lieu de 1995 pour le film Apollo 13.  
Conversion en int.

In [ ]:
df["Released_Year"] = df["Released_Year"].str.replace("PG","1995")

<ipython-input-168-b53c9f37b45a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Released_Year"] = df["Released_Year"].str.replace("PG","1995")


In [ ]:
df["Released_Year"] = df["Released_Year"].astype("int")

<ipython-input-169-e38671cf9a82>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Released_Year"] = df["Released_Year"].astype("int")


## Genres
Conversion en liste puis application d'un `MultiLabelBinarizer` sur le résultat.  
Son rôle est de faire un One Hot Encoding à partir d'un ensemble de listes de catégories (et non d'un ensemble de catégories). Exemple:
```
Liste Genre
["Action", "Drame"]
["Action", "Aventure"]
```
Résultat après le `MultiLabelBinarizer`:  
```
Action | Aventure | Drame  
   1   |     0    |   1  
   1   |     1    |   0  
```

In [ ]:
df["Liste Genre"] = df["Genre"].str.split(", ")

<ipython-input-170-a182897017a4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Liste Genre"] = df["Genre"].str.split(", ")


In [ ]:
mlb = MultiLabelBinarizer()

In [ ]:
df_genres = mlb.fit_transform(df["Liste Genre"])

In [ ]:
df = pd.concat([df, pd.DataFrame(df_genres, columns=mlb.classes_)], axis=1)

In [ ]:
df = df.drop(["Genre", "Liste Genre"], axis=1)

# Variables catégorielles : "Director", "Star1", "Star2", "Star3", "Star4"
On applique un `OneHotEncoder`, en laissant les autres colonnes intactes.  On impose une sortie sous forme de DataFrame pour conserver le nom des colonnes.

In [ ]:
col_cat = ["Director", "Star1", "Star2", "Star3", "Star4"]

In [ ]:
ct = ColumnTransformer(transformers = [
    ("cat", OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False), col_cat)
    ],
    remainder="passthrough",
    verbose_feature_names_out=False # Supprime le préfixe "cat_" et "remainder_" dans le nom des colonnes
)

In [ ]:
ct.set_output(transform="pandas")

ColumnTransformer(remainder='passthrough',
                  transformers=[('cat',
                                 OneHotEncoder(drop='first',
                                               handle_unknown='ignore',
                                               sparse_output=False),
                                 ['Director', 'Star1', 'Star2', 'Star3',
                                  'Star4'])],
                  verbose_feature_names_out=False)

In [ ]:
df_final = ct.fit_transform(df)

In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Columns: 3901 entries, Director_Aaron Sorkin to Western
dtypes: float64(3877), int64(24)
memory usage: 29.8 MB


In [ ]:
# Affichage des colonnes produites :
for i in df_final.columns:
  print(i)

Director_Aaron Sorkin
Director_Abdellatif Kechiche
Director_Abhishek Chaubey
Director_Abhishek Kapoor
Director_Adam Elliot
Director_Adam McKay
Director_Aditya Chopra
Director_Aditya Dhar
Director_Akira Kurosawa
Director_Alain Resnais
Director_Alan J. Pakula
Director_Alan Parker
Director_Alejandro Amenábar
Director_Alejandro G. Iñárritu
Director_Alejandro Jodorowsky
Director_Alex Garland
Director_Alex Proyas
Director_Alexander Mackendrick
Director_Alexander Payne
Director_Alfonso Cuarón
Director_Alfonso Gomez-Rejon
Director_Alfred Hitchcock
Director_Amit Ravindernath Sharma
Director_Anders Thomas Jensen
Director_Andrei Tarkovsky
Director_Andrew Adamson
Director_Andrew Davis
Director_Andrew Lau
Director_Andrew Niccol
Director_Andrew Stanton
Director_Andrey Zvyagintsev
Director_Aneesh Chaganty
Director_Ang Lee
Director_Aniruddha Roy Chowdhury
Director_Anthony Harvey
Director_Anthony Russo
Director_Antoine Fuqua
Director_Anton Corbijn
Director_Anurag Basu
Director_Anurag Kashyap
Director_A

# Imputation
On impute ici avec un `KNNImputer`.

In [ ]:
imputer = KNNImputer(n_neighbors=10)

In [ ]:
imputer.set_output(transform="pandas")

KNNImputer(n_neighbors=10)

In [ ]:
df_final = imputer.fit_transform(df_final)

In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Columns: 3901 entries, Director_Aaron Sorkin to Western
dtypes: float64(3901)
memory usage: 29.8 MB


# Normalisation
Pour attribuer autant de poids à chaque variable lors de la recherche des plus proches voisins, il faut normaliser tout le jeu de données.

In [ ]:
scaler = MinMaxScaler()
scaler.set_output(transform="pandas")

MinMaxScaler()

In [ ]:
df_final = scaler.fit_transform(df_final)

# Recommandation Content based avec les plus proches voisins

In [ ]:
model = NearestNeighbors(n_neighbors=10)

In [ ]:
model.fit(df_final)

NearestNeighbors(n_neighbors=10)

In [ ]:
# Essai avec le premier film du jeu de données :
indice_film_1 = 1
film_1 = df_final.iloc[indice_film_1]

In [ ]:
film_1

,1
Director_Aaron Sorkin,0.0
Director_Abdellatif Kechiche,0.0
Director_Abhishek Chaubey,0.0
Director_Abhishek Kapoor,0.0
Director_Adam Elliot,0.0
...,...
Sci-Fi,0.0
Sport,0.0
Thriller,0.0
War,0.0


In [ ]:
# distance, neig = model.kneighbors([film_1], n_neighbors=3)

In [ ]:
distance, neig = model.kneighbors(pd.concat([pd.DataFrame(), film_1.to_frame().T]), n_neighbors=10)

In [ ]:
distance

array([[0.        , 2.46875203, 3.08708885, 3.15440113, 3.1782059 ,
        3.18729972, 3.20966423, 3.24036526, 3.24077514, 3.26834148]])

In [ ]:
neig

array([[  1,   3, 974, 305,   6, 484,   4, 111,  22, 108]])

In [ ]:
print("Film initial : ")
print(df_0.iloc[indice_film_1])

for i in range(1, len(distance[0])):
  titre = df_0.iloc[neig[0][i]]["Series_Title"]
  print(f"{titre} recommandé à {100/(1+distance[0][i]):.2f}%") # On calcule la similarité à partir de la distance

  # print(df_0.iloc[neig[0][i]])

Film initial : 
Poster_Link      https://m.media-amazon.com/images/M/MV5BM2MyNj...
Series_Title                                         The Godfather
Released_Year                                                 1972
Certificate                                                      A
Runtime                                                    175 min
Genre                                                 Crime, Drama
IMDB_Rating                                                    9.2
Overview         An organized crime dynasty's aging patriarch t...
Meta_score                                                   100.0
Director                                      Francis Ford Coppola
Star1                                                Marlon Brando
Star2                                                    Al Pacino
Star3                                                   James Caan
Star4                                                 Diane Keaton
No_of_Votes                                   

In [ ]:
df_0[df_0["Series_Title"]=="Saw"]

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
932,https://m.media-amazon.com/images/M/MV5BMzQ2ZT...,Saw,2004,UA,103 min,"Horror, Mystery, Thriller",7.6,Two strangers awaken in a room with no recolle...,46.0,James Wan,Cary Elwes,Leigh Whannell,Danny Glover,Ken Leung,379020,"56,000,369"


In [ ]:
df_0["Series_Title"]

,Series_Title
0,The Shawshank Redemption
1,The Godfather
2,The Dark Knight
3,The Godfather: Part II
4,12 Angry Men
...,...
995,Breakfast at Tiffany's
996,Giant
997,From Here to Eternity
998,Lifeboat
